# 第3章 関数

In [24]:
from __future__ import annotations
from typing import TypeVar, Any

## 関数の戻り値が複数ある場合も、アスタリスク付き引数が使用できる

以下の関数とリストを例として使用します。

In [2]:
x = [.32, .24, .16, .08, .04, .02, .01]

In [3]:
def calc_deviation(x: list[float]) -> list[float]:
    """シーケンス要素の偏差を計算し、偏差の大きさでソートした新しいリストを返す"""
    
    if not x:
        raise ValueError("x must not be empty")
    
    x_mean = sum(x) / len(x)
    x_dev = [round(x_i - x_mean, ndigits=3) for x_i in x]    
    
    return sorted(x_dev)

ここで、偏差が最も大きい値のみを個別に取り出し、残りのリストと区別したいとしましょう。

**偏差が最も大きい値**と**それ以外のリスト**というアンパックを行うには、第2章で学んだアスタリスク付き引数を使用します。

In [4]:
largest, *others = calc_deviation(x)

print(largest, others)
# -0.114 [-0.104, -0.084, -0.044, 0.036, 0.116, 0.196]

-0.114 [-0.104, -0.084, -0.044, 0.036, 0.116, 0.196]


## 関数の戻り値が多数ある場合には`namedtuple`を使用する

それでは次に、上記の関数を改変して新たな例を作成します。

上記の偏差値は1つのシーケンスを返す関数でしたが、それに加えて、

シーケンス要素の偏差値が**最も大きい値と最も小さい値は個別の戻り値として返す**ほか、

追加で**平均値や中央値も計算して戻り値として返す**ようにしてみましょう。

標準ライブラリの`statistics`を使用します。

In [5]:
import statistics

In [6]:
def calc_statistics(x: list[float]) -> tuple[list[float], float, float, float, float]:
    """シーケンス要素の様々な統計値を計算
    
    Parameters
    ----------
    x : list[float]
        統計値を計算したいシーケンス
    
    Returns
    -------
    list[float]
        シーケンス各要素の偏差値のリスト
    float
        シーケンス各要素の偏差値のうち最大値
    float
        シーケンス各要素の偏差値のうち最小値
    float
        シーケンス各要素の平均値
    float
        シーケンス各要素の中央値
    """
    
    if not x:
        raise ValueError("x must not be empty")
    
    # 平均値
    x_mean = statistics.mean(x)
    
    # 偏差値のリスト
    x_dev = [round(x_i - x_mean, ndigits=3) for x_i in x]
    
    # 偏差値の最大値と最小値
    x_dev_max = max(x_dev)
    x_dev_min = min(x_dev)  
    
    # 中央値
    x_median = statistics.median(x)
    
    return x_dev, x_dev_max, x_dev_min, x_mean, x_median

戻り値が5つもある関数になってしまいました。素直にアンパックすると、以下のようになります。

In [7]:
x_dev, x_dev_max, x_dev_min, x_mean, x_median = calc_statistics(x)

(出力結果)

In [8]:
print(
    f"deviations = {x_dev}, \n",
    f"max value in deviations = {x_dev_max}, \n",
    f"min value in deviations = {x_dev_min}, \n", 
    f"mean value = {x_mean}, \n", 
    f"median value = {x_median}",
)
# deviations = [0.196, 0.116, 0.036, -0.044, -0.084, -0.104, -0.114], 
#  max value in deviations = 0.196, 
#  min value in deviations = -0.114, 
#  mean value = 0.12428571428571429, 
#  median value = 0.08

deviations = [0.196, 0.116, 0.036, -0.044, -0.084, -0.104, -0.114], 
 max value in deviations = 0.196, 
 min value in deviations = -0.114, 
 mean value = 0.12428571428571429, 
 median value = 0.08


このやり方は、各戻り値の位置を間違えてアンパックしてしまう危険があるために好ましくありません。

In [9]:
# x_dev_min と x_dev_max の位置を間違えた！
x_dev, x_dev_min, x_dev_max, x_mean, x_median = calc_statistics(x)

こういった場合には、標準ライブラリの`collections`から`namedtuple`を使用します。

`namedtuple`の素晴らしいところは以下の2つです。

1. 要素にインデックスではなく名称でアクセスでき、分かりやすい
1. `pandas.DataFrame`のようにドットアクセスができる (要するに、`tup["hoge"]`ではなく`tup.hoge`で要素にアクセスする)

==

それでは、`namedtuple`を使用して関数を書き直してみましょう(Docstringは省略)。

In [10]:
from collections import namedtuple
from typing import NamedTuple

In [11]:
def calc_statistics(x: list[float]) -> NamedTuple:
    if not x:
        raise ValueError("x must not be empty")
    
    # 平均値
    x_mean = statistics.mean(x)
    
    # 偏差値のリスト
    x_dev = [round(x_i - x_mean, ndigits=3) for x_i in x]
    
    # 偏差値の最大値と最小値
    x_dev_max = max(x_dev)
    x_dev_min = min(x_dev)  
    
    # 中央値
    x_median = statistics.median(x)
    
    # 各属性名を設定して、namedtupleを作成
    Stats = namedtuple('Stats', ['deviations', 'max', 'min', 'mean', 'median'])
    
    # namedtupleに値をセット
    x_stats = Stats(x_dev, x_dev_max, x_dev_min, x_mean, x_median)
    
    return x_stats

`namedtuple`を使用することで、非常に分かりやすいアンパックが実現できました。

In [12]:
x_stats = calc_statistics(x)

print(x_stats)
# Stats(deviations=[0.196, 0.116, 0.036, -0.044, -0.084, -0.104, -0.114], max=0.196, min=-0.114, mean=0.12428571428571429, median=0.08)

Stats(deviations=[0.196, 0.116, 0.036, -0.044, -0.084, -0.104, -0.114], max=0.196, min=-0.114, mean=0.12428571428571429, median=0.08)


先述したように、ドットアクセスで個々の要素にアクセスできます。

In [13]:
print(x_stats.median)  # 0.08

0.08


ちなみに、Effective Python 第2版には`namedtuple`がデフォルト値を設定できないことが欠点だと記載してありますが、**完全な誤り**です。

`typing.NamedTuple`を継承したクラスを作成することでデフォルト値が設定できます。

In [14]:
from typing import NamedTuple, Optional

class DataTable(NamedTuple):
    name: str
    value: float
    unit: Optional[str] = None
    
data_table = DataTable(name='x', value=0.2)

print(data_table)  # DataTable(name='x', value=0.2, unit=None)

DataTable(name='x', value=0.2, unit=None)


しかし、`namedtuple`には`tuple`らしくイミュータブルという特性があります。

もちろん、好ましい特性でもありますが、動的に値を変更したい場合にはより柔軟性の高い`collections.dataclasses`の`dataclass`デコレータを付与したクラスを使用した方が良さそうです。

* `collections.dataclasses.dataclass`の参考サイトは[こちら](https://zenn.dev/enven/articles/8b80ff38461b4ff329aa)

## 関数に動的なデフォルト引数を指定したい場合は、一旦`None`としておく

以下の関数を例とします。

In [15]:
from datetime import datetime
def say_hello_and_show_date(name: str, d: datetime = datetime.now()) -> str:
    return f"こんにちは、{name}-san. " + f"現在の日時は{str(d)}です。"

`d`のデフォルト引数が`datetime.now()`になっているので、関数実行時の日時が表示されることが期待できます。

それでは、実行してみましょう。

In [16]:
say_hello_and_show_date(name="太郎")
# 'こんにちは、太郎-san. 現在の日時は2022-04-24 16:02:32.529558です。'

'こんにちは、太郎-san. 現在の日時は2022-04-24 16:08:22.424497です。'

もう一回実行してみます。

In [17]:
say_hello_and_show_date(name="太郎")
# 'こんにちは、太郎-san. 現在の日時は2022-04-24 16:02:32.529558です。'

'こんにちは、太郎-san. 現在の日時は2022-04-24 16:08:22.424497です。'

お気づきのように、日時部分が2回の実行で一切変わっていません。

これは、`datetime.now()`が関数評価時の1回しか実行されないために起きる問題です。

回避するためには、デフォルト引数を`None`とし、関数内で実際の振る舞いを定義します。

In [18]:
def say_hello_and_show_date(name: str, d: Optional[datetime] = None) -> str:
    """あいさつと現在の日時を返す

    Parameters
    ----------
    name : str
        名前
    d : Optional[datetime], optional
        呼び出し側で値が指定されていない場合、
        呼び出された日時を返します, by default None

    Returns
    -------
    str
        あいさつと現在の日時
    """
    
    # ここで実際の振る舞いを定義
    if d is None:
        d = datetime.now()
    
    return f"こんにちは、{name}-san. " + f"現在の日時は{str(d)}です。"

デフォルト引数が`None`をとっていることに対しての説明をDocstringやコメントに記載しておくと、後々コードを読むことになる人にも親切です。

それでは、2回実行してみます。

In [19]:
say_hello_and_show_date(name="太郎")
# 'こんにちは、太郎-san. 現在の日時は2022-04-24 16:02:37.311410です。'

'こんにちは、太郎-san. 現在の日時は2022-04-24 16:08:22.557143です。'

In [20]:
say_hello_and_show_date(name="太郎")
# 'こんにちは、太郎-san. 現在の日時は2022-04-24 16:03:13.593031です。'

'こんにちは、太郎-san. 現在の日時は2022-04-24 16:08:22.590356です。'

問題なく動作することが分かりました。

## キーワード専用引数・位置専用引数を指定して明確な呼び出しを義務付ける

以下に定義する、式を積分・微分する関数を例に考えます。

In [50]:
import sympy
from sympy import sympify

In [114]:
def calculator(
    str_expr: str, 
    what_to_do: str, 
    by: str | int | float, 
    limit: Optional[str | int] = "oo",
    limit_dir: Optional[str] = "+-",
    return_type: str = "expr"
) -> sympy.Expr | str:
    """計算式を計算して返す
    
    Parameters
    ----------
    str_expr : str
        計算式
    what_to_do : str
        計算する内容
    by : str | int | float
        独立変数
    limit : str | int, optional
        極限値を得たい任意の点, by default 'oo'
    limit_dir : str, optional
        左側極限を得たい場合は'-', 右側極限の場合は'+', by default '+-'
    return_type : str, optional
        返す式の型, by default 'expr'
    
    Returns
    -------
    Sympy.Expr | str
        計算結果
        
    Raises
    ------
    ValueError
        引数の値が不正な場合
    """
    
    if what_to_do not in ["limit", "diff", "integrate"]:
        raise ValueError("what_to_doの値は'limit', 'diff', 'integrate'のみ指定できます。")
    
    # 式を評価
    expr = sympify(str_expr)
    calc_by = sympy.Symbol(by)
    
    res = calc_helper(expr, what_to_do, calc_by, limit, limit_dir)
    
    if return_type == "expr":
        return res
    elif return_type == "str":
        return str(res)
    else:
        raise ValueError("return_typeは'expr'または'str'のみ指定できます。")

In [115]:
def calc_helper(expr, what_to_do, calc_by, *args) -> sympy.Expr:
    if what_to_do == "limit":
        return sympy.limit(expr, calc_by, *args)
    elif what_to_do == "diff":
        return sympy.diff(expr, calc_by)
    elif what_to_do == "integrate":
        return sympy.integrate(expr, calc_by)
    else:
        raise ValueError("what_to_doの値が不正です。")

まずは実行してみます。

In [116]:
calculator("x**3+2*x**2+6*x", "diff", "x")
# '3*x**2 + 4*x + 6'

3*x**2 + 4*x + 6

In [117]:
calculator("x**3+2*x**2+6*x", "integrate", "x")
# 'x**4/4 + 2*x**3/3 + 3*x**2'

x**4/4 + 2*x**3/3 + 3*x**2

In [118]:
calculator("1/x", "limit", "x", 0, "+")
# 'oo'

oo

問題は出ていませんが、関数呼び出しの際に指定する引数が多いために、引数の順番が気がかりになってきます。

そこで関数定義の際、全ての引数の前に`*`を記載することによって、引数をキーワード専用引数とすることができます。

In [127]:
def modified_calculator(
    *,  # 全ての引数をキーワード専用引数に設定
    str_expr: str, 
    what_to_do: str, 
    by: str | int | float, 
    limit: Optional[str | int] = "oo",
    limit_dir: Optional[str] = "+-",
    return_type: str = "expr"
) -> sympy.Expr | str:   
    
    if what_to_do not in ["limit", "diff", "integrate"]:
        raise ValueError("what_to_doの値は'limit', 'diff', 'integrate'のみ指定できます。")
    
    # 式を評価
    expr = sympify(str_expr)
    calc_by = sympy.Symbol(by)
    
    res = calc_helper(expr, what_to_do, calc_by, limit, limit_dir)
    
    if return_type == "expr":
        return res
    elif return_type == "str":
        return str(res)
    else:
        raise ValueError("return_typeは'expr'または'str'のみ指定できます。")

キーワードなしに位置で引数を指定しようとすると怒られることが分かります。

In [137]:
try:
    modified_calculator("1/x", "limit", "x", "oo")
    print("OK!")
except TypeError:
    print("ダメです。")
    
# ダメです。

ダメです。


ですので、全ての引数にキーワードを設定すると上手くいきます。

In [138]:
try:
    modified_calculator(str_expr="1/x", what_to_do="limit", by="x", limit="oo")
    print("OK!")
except TypeError:
    print("ダメです。")
    
# OK!

OK!


とはいえ、第1引数で式・第2引数で計算内容を指定するのは直感に適っていますので、そこにまでキーワードの指定を強要するのは少し冗長です。

思うに、キーワード引数にしたいような紛らわしい引数は第3引数以降のみではないでしょうか。

このように、位置引数とキーワード専用引数を区別して定義したいときには`/`と`*`を組み合わせます。

In [131]:
def nice_calculator(
    str_expr: str, 
    what_to_do: str, 
    /,  # ここまでの引数を位置引数に設定
    *,  # これ以降の引数をキーワード専用引数に設定
    by: str | int | float, 
    limit: Optional[str | int] = "oo",
    limit_dir: Optional[str] = "+-",
    return_type: str = "expr"
) -> sympy.Expr | str:   
    
    if what_to_do not in ["limit", "diff", "integrate"]:
        raise ValueError("what_to_doの値は'limit', 'diff', 'integrate'のみ指定できます。")
    
    # 式を評価
    expr = sympify(str_expr)
    calc_by = sympy.Symbol(by)
    
    res = calc_helper(expr, what_to_do, calc_by, limit, limit_dir)
    
    if return_type == "expr":
        return res
    elif return_type == "str":
        return str(res)
    else:
        raise ValueError("return_typeは'expr'または'str'のみ指定できます。")

キーワードなしに位置で指定したい引数を書いた後に`/`、キーワードの指定を義務付ける引数の前に`*`を置くことで、位置専用引数とキーワード専用引数を分けることができます。

In [139]:
nice_calculator("1/x", "limit", by="x", limit="oo")

0

この状態だと、逆に位置専用引数に対してキーワードを指定すると怒られます。

In [142]:
try:    
    nice_calculator(str_expr="1/x", what_to_do="limit", by="x", limit="oo")
    print("OK!")
except TypeError as e:
    print(e)
    
# nice_calculator() got some positional-only arguments passed as keyword arguments: 'str_expr, what_to_do'

nice_calculator() got some positional-only arguments passed as keyword arguments: 'str_expr, what_to_do'


位置で指定しても、キーワードで指定しても良い引数を設定したい場合には、`/`と`*`の間に定義します。

In [145]:
def better_calculator(
    str_expr: str, 
    /,
    what_to_do: str, # "/"と"*"の間の引数は位置・キーワードどちらでも指定できる
    *,
    by: str | int | float, 
    limit: Optional[str | int] = "oo",
    limit_dir: Optional[str] = "+-",
    return_type: str = "expr"
) -> sympy.Expr | str:   
    
    if what_to_do not in ["limit", "diff", "integrate"]:
        raise ValueError("what_to_doの値は'limit', 'diff', 'integrate'のみ指定できます。")
    
    # 式を評価
    expr = sympify(str_expr)
    calc_by = sympy.Symbol(by)
    
    res = calc_helper(expr, what_to_do, calc_by, limit, limit_dir)
    
    if return_type == "expr":
        return res
    elif return_type == "str":
        return str(res)
    else:
        raise ValueError("return_typeは'expr'または'str'のみ指定できます。")

この例だと、`str_expr`は位置専用の引数で、`by`以降がキーワード専用引数となりますが、

`what_to_do`は位置・キーワードどちらで指定しても問題のない引数になります。

要するに`/`と`*`の間の引数は**Pythonにおいて通常の引数**です。

In [146]:
try:    
    nice_calculator("1/x", what_to_do="limit", by="x", limit="oo")
    print("OK!")
except TypeError as e:
    print(e)
    
# OK!

OK!


In [147]:
try:    
    nice_calculator("1/x", "limit", by="x", limit="oo")
    print("OK!")
except TypeError as e:
    print(e)
    
# OK!

OK!
